In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import math
import seaborn as sns

In [ ]:
df = pd.read_csv("../input/flight-take-off-data-jfk-airport/M1_final.csv")
print(df)

In [ ]:
df.info()

MISSING DATA

In [ ]:
sns.heatmap(df.isnull(),yticklabels=False,cbar=False,cmap='viridis')

Don't know how much data is missing

In [ ]:
df.isnull().sum()

Since we have larger data set so we can drop two rows easily

In [ ]:
df = df.dropna()
df.isnull().sum()
print('Dataframe dimensions:', df.shape)
df.head()

In [ ]:
df.drop('TAIL_NUM',axis=1,inplace=True)
df.head()

**HISTOGRAM PLOTS**

In [ ]:
df.hist(bins = 50, figsize = (40, 30))
plt.show()

Histogram plots provides some idea of the distribution of independent variables (numerical and dates).
(1). Most of the 'Wind Gust' values are near 0. 
(2). Not all the values of the column 'Dew Point' is Numerical. some of them are in string format. This is why its histogram is not shown. 
(3). Cetegorical variables are - 'OP_UNIQUE_CARRIER', 'DEST', 'Dew Point', 'Wind', 'Condition' (We have to apply Label and one hot encoding both on these variables)

**We have to handle 'Due Point' column**

In [ ]:
df = df[df['Dew Point'].map(lambda x: x.isascii())]
print('Dataframe dimensions:', df.shape)

**Input and output variable in Dataframe**

In [ ]:
X = df.drop(['TAXI_OUT'],axis = 1)
Y = df.TAXI_OUT
print('Input dimensions:', X.shape)
print('Output dimensions:', Y.shape)
X.head()

**Train test splitting**

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.1)
print('Input training dimensions:', X_train.shape)
print('Input testing dimensions:', X_test.shape)
print('Output training dimensions:', Y_train.shape)
print('Output testing dimensions:', Y_test.shape)

# 1.Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
X1_train = X_train.copy()
X1_test = X_test.copy()
X1_train['OP_UNIQUE_CARRIER'] =labelencoder.fit_transform(X1_train['OP_UNIQUE_CARRIER'].astype(str))
X1_test['OP_UNIQUE_CARRIER'] =labelencoder.fit_transform(X1_test['OP_UNIQUE_CARRIER'].astype(str))
X1_train['DEST'] = labelencoder.fit_transform(X1_train['DEST'].astype(str))
X1_train['Dew Point'] = labelencoder.fit_transform(X1_train['Dew Point'].astype(str))
X1_train['Wind'] = labelencoder.fit_transform(X1_train['Wind'].astype(str))
X1_train['Condition'] = labelencoder.fit_transform(X1_train['Condition'].astype(str))
X1_test['DEST'] = labelencoder.fit_transform(X1_test['DEST'].astype(str))
X1_test['Dew Point'] = labelencoder.fit_transform(X1_test['Dew Point'].astype(str))
X1_test['Wind'] = labelencoder.fit_transform(X1_test['Wind'].astype(str))
X1_test['Condition'] = labelencoder.fit_transform(X1_test['Condition'].astype(str))
X1_train.head()

**Training and pridicting**

In [ ]:
from sklearn.metrics import mean_squared_error

**A.LINEAR REGRESSION**

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X1_train,Y_train)
Y_predicted1 = linear_reg.predict(X1_test)
l1 = mean_squared_error(Y_predicted1,Y_test,squared = False)
print('Linear Regression Error: ', l1)

**B.RIDGE REGRESSION**

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X1_train,Y_train)
Y_predicted2 = ridge_reg.predict(X1_test)
l2 = mean_squared_error(Y_predicted2, Y_test, squared = False)
print('Ridge Regression Error: ', l2)

**C. LASSO REGRESSION**

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso()
lasso_reg.fit(X1_train,Y_train)
Y_predicted3 = lasso_reg.predict(X1_test)
l3 = mean_squared_error(Y_predicted3, Y_test, squared = False)
print('Lasso Regression Error: ', l3)

**D. KNN MODEL**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(n_neighbors = 200)
knn_reg.fit(X1_train,Y_train)
Y_predicted4=knn_reg.predict(X1_test)
l4 = mean_squared_error(Y_predicted4,Y_test, squared =False)
print('KNN Model Error: ', l4)

**E. SVR**

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
x1_train = X1_train.copy()
x1_test = X1_test.copy()
s1 = StandardScaler()
s2 = StandardScaler()
x1_train = s1.fit_transform(x1_train)
x1_test = s2.fit_transform(x1_test)
sv_reg  = SVR()
sv_reg.fit(x1_train,Y_train)
Y_predicted5 = sv_reg.predict(x1_test)
l5 = mean_squared_error(Y_predicted5, Y_test, squared= False)
print('SVR Error: ', l5)

**F. NAIVE BAYES**

In [ ]:
from sklearn.linear_model import BayesianRidge
bayes_reg = BayesianRidge()
bayes_reg.fit(X1_train, Y_train)
Y_predicted6 = bayes_reg.predict(X1_test)
l6 = mean_squared_error(Y_predicted6, Y_test,squared=False)
print('Naive Bayes Error:', l6)

**G. RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
random_for = RandomForestRegressor()
random_for.fit(X1_train, Y_train)
Y_predicted7 = random_for.predict(X1_test)
l7 = mean_squared_error(Y_predicted7, Y_test, squared = False)
print('Random Forest Error:', l7)

**H. LightBGM**

In [ ]:
from lightgbm import LGBMRegressor
light_reg = LGBMRegressor()
light_reg.fit(X1_train,Y_train)
Y_predicted8 = light_reg.predict(X1_test)
l8 = mean_squared_error(Y_predicted8, Y_test,squared=False)
print('LightGBM Error:', l8)

# 2. ONE-HOT ENCODING

One_Hot encoding only for 'OP_UNIQUE_CARRIER','Wind','Condition' variables

Label encoding for DEST and Dew Point because unique values for them are more than 50.

In [ ]:
X2 = X.copy()
X2 = pd.get_dummies(X2, columns = ['OP_UNIQUE_CARRIER','Wind','Condition'])
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
X2['DEST']=label_encoder.fit_transform(X2['DEST'].astype(str))
X2['Dew Point']=label_encoder.fit_transform(X2['Dew Point'])
from sklearn.model_selection import train_test_split
X2_train,X2_test,Y_train,Y_test = train_test_split(X2,Y,test_size = 0.1)
print('Input training dimensions:', X2_train.shape)
print('Input testing dimensions:', X2_test.shape)
print('Output training dimensions:', Y_train.shape)
print('Output testing dimensions:', Y_test.shape)

**Training and predicting**

In [ ]:
from sklearn.metrics import mean_squared_error

**LINEAR REGRESSION**

In [ ]:
from sklearn.linear_model import LinearRegression
linear_reg = LinearRegression()
linear_reg.fit(X2_train,Y_train)
Y_predicted1 = linear_reg.predict(X2_test)
h1 = mean_squared_error(Y_predicted1,Y_test,squared = False)
print('Linear Regression Error: ', h1)

**RIDGE REGRESSION**

In [ ]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge()
ridge_reg.fit(X2_train,Y_train)
Y_predicted2 = ridge_reg.predict(X2_test)
h2 = mean_squared_error(Y_predicted2, Y_test, squared = False)
print('Ridge Regression Error: ', h2)

**LASSO REGRESSION**

In [ ]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso()
lasso_reg.fit(X2_train,Y_train)
Y_predicted3 = lasso_reg.predict(X2_test)
h3 = mean_squared_error(Y_predicted3, Y_test, squared = False)
print('Lasso Regression Error: ', h3)

**KNN MODEL**

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor(n_neighbors = 200)
knn_reg.fit(X2_train,Y_train)
Y_predicted4=knn_reg.predict(X2_test)
h4 = mean_squared_error(Y_predicted4,Y_test, squared =False)
print('KNN Model Error: ', h4)

**SVR**

In [ ]:
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
x2_train = X2_train.copy()
x2_test = X2_test.copy()
s1 = StandardScaler()
s2 = StandardScaler()
x2_train = s1.fit_transform(x2_train)
x2_test = s2.fit_transform(x2_test)
sv_reg  = SVR()
sv_reg.fit(x2_train,Y_train)
Y_predicted5 = sv_reg.predict(x2_test)
h5 = mean_squared_error(Y_predicted5, Y_test, squared= False)
print('SVR Error: ', h5)

**NAIVE BAYES**

In [ ]:
from sklearn.linear_model import BayesianRidge
bayes_reg = BayesianRidge()
bayes_reg.fit(X2_train, Y_train)
Y_predicted6 = bayes_reg.predict(X2_test)
h6 = mean_squared_error(Y_predicted6, Y_test,squared=False)
print('Naive Bayes Error:', h6)

**RANDOM FOREST**

In [ ]:
from sklearn.ensemble import RandomForestRegressor
random_for = RandomForestRegressor()
random_for.fit(X2_train, Y_train)
Y_predicted7 = random_for.predict(X2_test)
h7 = mean_squared_error(Y_predicted7, Y_test, squared = False)
print('Random Forest Error:', h7)

**LightBGM**

In [ ]:
from lightgbm import LGBMRegressor
light_reg = LGBMRegressor()
light_reg.fit(X2_train,Y_train)
Y_predicted8 = light_reg.predict(X2_test)
h8 = mean_squared_error(Y_predicted8, Y_test,squared=False)
print('LightGBM Error:', h8)

**COMPARISON AND CONCLUSION**

Label Encoding Errors

In [ ]:
print(l1, l2, l3, l4, l5, l6, l7, l8)

One-Hot Encoding Errors

In [ ]:
print(h1, h2, h3, h4, h5, h6, h7, h8)

In [ ]:
from matplotlib import style
label_errors = np.array([l1,l2,l3,l4,l5,l6,l7,l8])
onehot_errors = np.array([h1,h2,h3,h4,h5,h6,h7,h8])
style.use("ggplot")
plt.plot(["linear","Ridge","Lasso","KNN","SVR","BR","RanFor","LGBM"],label_errors, "g",label="Label Encoding")
plt.plot(["linear","Ridge","Lasso","KNN","SVR","BR","RanFor","LGBM"],onehot_errors, "b",label="One Hot Encoding")
plt.legend(["Label Encoding","One Hot Encoding"])
plt.xlabel("ML Models")
plt.ylabel("RMSE")
plt.show

1. One hot encoding gives good predictions.
2. Random Forest is best out of all 8 algorithms for prediction analysis.